# Day 8

### Topics

1. Bit of a review: widgets & `ipywidgets.interact`
1. More about widgets
  * Bonus: Linking widgets
1. The NASA planet archive data
1. `bqplot` heatmap with planet data
  * Bonus: `bqplot` dashboard



First, import our usual things:

In [1]:
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import bqplot.pyplot as bplt

## Bit of a review

Last time we got into some interactivity with `ipywidgets` and the `interact` decorator:

There are a few ways we can use this library in a few ways, the first way is through something called a "decorator" that will decorate a function.  The syntax will look a little like:

```python
@decorator
def my_function():
    # stuff
...
```

This decorator is "decorating" the function like you'd put a hat to decorate your head -- it sits on top of the function and interacts only with this function.  Let's try a simple one to start -- print out a phrase from a list.

Let's sketch out what we want to do with code statically and build from there.  First, let's make something that prints a phrase:

In [2]:
print('Hi there!')

Hi there!


Now, let's make the actual phrase an input:

In [3]:
phrase = 'I am phrase.'

print(phrase)

I am phrase.


Now, let's make this into a function:

In [4]:
def my_phrase_print(phrase):
    print(phrase)

Let's use it a few times:

In [5]:
my_phrase_print('hi!')

hi!


In [6]:
my_phrase_print('hi hi!')
my_phrase_print('helooooooo!!')

hi hi!
helooooooo!!


Now let's decorate this function with the decorator `@ipywidgets.interact`:

In [7]:
@ipywidgets.interact(phrase='')
def my_phrase_print(phrase):
    print(phrase)

interactive(children=(Text(value='', description='phrase'), Output()), _dom_classes=('widget-interact',))

Whoa!  What did we just do!  We told `ipywidgets` that our phrase was going to be text input which it figured out by the default value we put in its definition AND that the input parameter `phrase` was going to be the one we wanted to change.  Let's try another.  Also note that when we ran this cell it called the function where before we would have to call the function after we declared it.


Let's say we want to only let our user put in a selected set of phrases from a list:

In [8]:
phrases_list = ['hi there!', 'I am phrase.', 'decorators decorate!']

We could do the same thing again but with this list:

In [9]:
@ipywidgets.interact(phrase=phrases_list)
def my_phrase_print(phrase):
    print(phrase)

interactive(children=(Dropdown(description='phrase', options=('hi there!', 'I am phrase.', 'decorators decorat…

Now we have a dropdown menu instead!  There are ways to choose what kinds of widgets we get (integer sliders, dropdown menus, text boxes), but here we are having `ipywidgets` basically figure it out for us on its own.

## More about widgets

But now, lets go into ipywidgets in a bit more detail and look at specific functions and call them directly from ipywidgets.

For example, we can create a little display that increments integer numbers:

In [10]:
itext = ipywidgets.IntText()
itext

IntText(value=0)

But what if that didn't display for you?  Depending on what version of jupyter notebook you have, you might have to use "display" to actually show your widgets.  

Try (uncommenting) the following:

In [11]:
#from IPython.display import display
#display(itext)

Still didn't work?  Try refreshing and/or restarting your notebook.

*Still* not working?  You might have to install the jupyter notebook widget extension:

In [13]:
#!jupyter nbextension enable --py widgetsnbextension

If you end up running the above cell (uncommented naturally!) then you might have to refresh or restart your jupyter notebook.

Moving on: 

The value of `itext` is then stored - so we could in theory generate a toggle and then do stuff with its value:

In [14]:
itext.value

0

Note if I go up and change the toggle value I have to re-run this cell to print out the newly stored value.

I can also set the value "by hand":

In [15]:
itext.value = 10

Once I run this cell, now the toggle values are updated above.

Let's try another one, how about a button:

In [16]:
button = ipywidgets.Button(description = "I am a Clicker")
button

Button(description='I am a Clicker', style=ButtonStyle())

So, cool, we have a button but nothing happens when I press it.  We want something to happen!  This is where this call back sort of thing comes into play.  Let's look at all the things associated with this button object:

In [19]:
dir(button)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_click_handlers',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_handle_button_msg',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_property_lock',
 '_register_validator',
 '_remove_notifiers',
 '_re

So, there is a lot of stuff here!  Let's focus on the `.observe` function:

In [21]:
button.observe?

In [22]:
def clicks(change):
    print(change)

In [27]:
button.observe(clicks, 'value')

In [28]:
button

Button(description='I am a Clicker', style=ButtonStyle())

Ok, this has a lot of weird language about "callbacks" but the main translation of this is that this "on_click" button is *observing* the button widget object for a click and then doing something when that button is clicked.  Let's try associating